Playing with web3.py

Install Depedencies

In [44]:
import etherscan
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt # importing matplotlib
import numpy as np # importing numpy
import schedule
import time
import json
import requests

from dotenv import load_dotenv
from datetime import datetime
from web3 import Web3
from os import getenv
from tqdm import tqdm
from decimal import Decimal

Connecting to web3 using Infura

In [4]:
load_dotenv()


es = etherscan.Client(api_key=getenv('ETHERSCAN'))
w3 = Web3(Web3.HTTPProvider(getenv('INFURA')))

w3.isConnected()

True

In [89]:
# address book
carlisle = '0xd791427e372186E90637caF1D6D6AFb054BeeC7A'
rarible_contract = '0xcd4EC7b66fbc029C116BA9Ffb3e59351c20B5B06'
opensea = '0x7Be8076f4EA4A4AD08075C2508e481d6C946D12b'
foundation = '0xcDA72070E455bb31C7690a170224Ce43623d0B6f'
superrare = '0x41A322b28D0fF354040e2CbC676F0320d8c8850d'
knownorigin = '0x848b0EA643e5a352D78e2c0c12A2dD8c96feC639'

In [15]:
def get_balance(wallet):
    wei = w3.eth.getBalance(wallet)
    return w3.fromWei(wei, 'ether')

get_balance(carlisle)

Decimal('0.054485017212272068')

In [ ]:
block = w3.eth.getBlock('latest')
block.transactions

In [42]:
transaction = '0x7a04337a420fede66c1038c3288b250c8e1ffd3fd6de46a3ec44a09150d803b8'

contract = w3.eth.getTransaction(transaction)
contract.input


'0xa0712d6800000000000000000000000000000000000000000000000dcc988cf91e06d800'

In [128]:
rarible = pd.read_csv('rarible.csv')

In [130]:
def mp_transactions(address):
    columns = ['timestamp', 'block_number', 'from', 'to', 'input', 'hash', 'value', 'gas', 
                'gas_price', 'gas_used', 'nonce', 'confirmations', 'is_error', 'tx_receipt_status', 
                'transaction_index', 'cumulative_gas_used', 'block_hash']

    transactions = []

    start_block = rarible['block_number'].iloc[-1]

    for l in tqdm(es.get_transactions_by_address(address, start_block=start_block, limit=9999)):
        time.sleep(.2)
        transactions.append(l)

    df = pd.DataFrame(transactions, columns=columns)
    df['timestamp'] = df['timestamp'].apply(datetime.fromtimestamp)
    df = df[df['value'] != 0]
    df = df[df['is_error'] != True]
    df['value'] = [w3.fromWei(x, 'ether') for x in df['value']]
    df = df.reset_index(drop=True)
    df.to_csv('rarible.csv', header=None, mode='a')
    return df

In [131]:
mp_transactions(rarible_contract)

100%|██████████| 13/13 [00:02<00:00,  4.87it/s]


,timestamp,block_number,from,to,input,hash,value,gas,gas_price,gas_used,nonce,confirmations,is_error,tx_receipt_status,transaction_index,cumulative_gas_used,block_hash
0,2021-04-22 20:48:30,12293597,0x4cff7990f598a956d17ab11e80664bc8098d5df2,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000ee74c2b60465...,0xfe875e57019c7b7d2c61886fc4905af6055e7a232e9f...,0.03075,249715,93000000000,249715,106,312,False,True,146,13194548,0x37e64334304250da2213ea83ce92eded278e415b9edc...
1,2021-04-22 20:58:05,12293633,0xe4160a3bef14dfdc0cf49d9dcecd16ffe441de91,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec63920000000000000000000000003d05ad7a259b...,0x4b06592de7d36d8639ae6f2662c170d688ce9c954ab9...,0.0066625,249751,152000000000,249751,252,276,False,True,96,8205005,0x60c3a987a1a4e636a65de97e26681d98a848e97c2dae...
2,2021-04-22 21:01:27,12293652,0x235e02f78bbff1fab6d389e5510785304364fd91,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec639200000000000000000000000031457ac01f55...,0x31f458ee078b6c99ad0589ef7e8eee859f75c55b3c89...,0.15375,266827,120000000000,266827,0,257,False,True,115,10553176,0x4eebf11048cd7d52cfa1b5b7edcc25bf6acc16630d57...
3,2021-04-22 21:03:26,12293662,0xc390cb09c9474cb794e3fc14915064554f4f1ab6,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec63920000000000000000000000009a0297bc33d3...,0x3a4ca54fc3c815ef990ec8082b2d30ce4932825805a4...,0.1025,249727,125000000000,249727,0,247,False,True,226,12699801,0x3e8076eadbb794d97cdd964e390348f092e198598480...
4,2021-04-22 21:03:26,12293662,0x87ec261ba759ca3d66fcd8199f6fb9b36dc4badd,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000c3c9fdee83ad...,0x20e802d95f5ccd0051a9cd0ba185a933d679830ee608...,0.3075,342357,125000000000,312357,213,247,False,True,228,13156068,0x3e8076eadbb794d97cdd964e390348f092e198598480...
5,2021-04-22 21:03:46,12293664,0x8f7cce709fcaee2a78219df63f5fa05887427c50,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000f9c446998102...,0xdd8d063d7bdba2a2bce0e442dd1b210f1d2bbab86300...,0.0001025,269327,121000001459,254327,1,245,False,True,267,13711934,0x7a83edaee5ffaf1115b52b4bb791b6a14d44d6224a5e...
6,2021-04-22 21:32:56,12293799,0xa17fcac52659a2bd1f7d60da6adcc24321e41079,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000062b98dc590a...,0x86490581a7c59aa1b5d61b8a4620b9e28e933017a187...,0.1025,266803,212500000000,266803,24,110,False,True,90,11690551,0xe8b0a23bd05f2c32f50996a98ccb19b6bd9594c8a831...
7,2021-04-22 21:38:06,12293818,0xdfe279c336e767beb9fb1c19c1695e2000a2c720,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec63920000000000000000000000001f8e698c3179...,0x75756cc591784760fef21cce28bb2d42dae6f8b2e25f...,2.36775,269339,264000000000,269339,217,91,False,True,224,6486325,0x803a39652d5f4a8e4c743fefd3ebb335807cb56f25f3...
8,2021-04-22 21:52:18,12293875,0x2fa863a450cd5b82adf910d7ae8ac1de686d01a8,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec63920000000000000000000000001d2e139b693c...,0x863a749b454dd32d8d902e336823b9a9eca3de314a2d...,0.205,249727,243100000000,249727,320,34,False,True,124,10534252,0xd6280400a096dea4da0ed4d77f2d9a0dba5f8add2312...
9,2021-04-22 21:52:26,12293877,0x3ec3e0fc4623130fc978e9f02c4db9b10986e909,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000006222da945e...,0xd7c1e3b804150c27106b5717debc6b923048deb06ebd...,0.05125,249715,243100000000,249715,12,32,False,True,14,1360646,0x187b81f134424e649c13118e3637f510c86632ec944e...


In [127]:
rarible_df = pd.read_csv('rarible.csv')
rarible_df

ParserError: Error tokenizing data. C error: Expected 18 fields in line 2239, saw 35


In [115]:
fig = go.Figure()

value = rarible_df['value']
timestamp = rarible_df['timestamp']
a_hash = rarible_df['hash']
mean = rarible_df['value'].mean()

fig.add_trace(go.Scatter(y=value, x=timestamp, name=str(a_hash), mode='markers', text=rarible_df.hash,
                marker=dict(size=value.astype(float), color=value, colorscale='Cividis', showscale=True, line=dict(width=.5,
                                        color='grey'))))

fig.add_shape(type='line', x0=min(timestamp), y0=30, x1=max(timestamp), y1=30,
                line=dict(color='Red') ,xref='x', yref='y')

fig.update_layout(yaxis_title="", xaxis_title="<i>contract: 0xcd4EC7b66fbc029C116BA9Ffb3e59351c20B5B06</i>", width=600, height=900, yaxis=dict(zeroline=False),
                    template='seaborn', legend_title="Comparison", paper_bgcolor='rgba(0,0,0,0)', 
                    font=dict(family="Tahoma", size=10), plot_bgcolor='rgba(0,0,0,0)',
                    title=("Rarible<br>" +
                    "<i>Value of transactions including bids and sales</i>"))

# Add images
fig.add_layout_image(dict(source="gradient.png", xref="paper", yref="paper",
                        x=-1.5, y=1.2, sizex=6, sizey=6, layer="below"))

fig.add_annotation(x=rarible_df['timestamp'].iloc[2170], y=70.5,
            text="", showarrow=False, arrowhead=1)

fig.update_shapes(dict(xref='x', yref='y'))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update_traces(marker_showscale=False)
fig.show()

In [ ]:
def hist_for_count(data):
    plt.style.use("fast")
    plt.figure(figsize=(21,14), facecolor=('#fff9f4')) # Make it 14x7 inch

    x = data # normal distribution
    n, bins, patches = plt.hist(x, bins=80, facecolor='#2ab0ff', edgecolor='#e0e0e0', linewidth=0.5, alpha=0.7)
    n = n.astype('int') # it MUST be integer

    # Good old loop. Choose colormap of your taste
    for i in range(len(patches)):
        patches[i].set_facecolor(plt.cm.viridis(n[i]/max(n)))

    # Add title and labels with custom font sizes
    plt.title('', fontsize=12)
    plt.xlabel('', fontsize=8)
    plt.ylabel('', fontsize=8)
    plt.grid(b=None)
    plt.savefig('foo.png')
    plt.show();
    return

In [ ]:
hist_for_count(foundation_df['timestamp'])

In [4]:
# OMG Address
abi = json.loads('[{"constant":true,"inputs":[],"name":"mintingFinished","outputs":[{"name":"","type":"bool"}],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"_spender","type":"address"},{"name":"_value","type":"uint256"}],"name":"approve","outputs":[],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"_from","type":"address"},{"name":"_to","type":"address"},{"name":"_value","type":"uint256"}],"name":"transferFrom","outputs":[],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint256"}],"payable":false,"type":"function"},{"constant":false,"inputs":[],"name":"unpause","outputs":[{"name":"","type":"bool"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"_to","type":"address"},{"name":"_amount","type":"uint256"}],"name":"mint","outputs":[{"name":"","type":"bool"}],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"paused","outputs":[{"name":"","type":"bool"}],"payable":false,"type":"function"},{"constant":true,"inputs":[{"name":"_owner","type":"address"}],"name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],"payable":false,"type":"function"},{"constant":false,"inputs":[],"name":"finishMinting","outputs":[{"name":"","type":"bool"}],"payable":false,"type":"function"},{"constant":false,"inputs":[],"name":"pause","outputs":[{"name":"","type":"bool"}],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"owner","outputs":[{"name":"","type":"address"}],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"name":"","type":"string"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"_to","type":"address"},{"name":"_value","type":"uint256"}],"name":"transfer","outputs":[],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"_to","type":"address"},{"name":"_amount","type":"uint256"},{"name":"_releaseTime","type":"uint256"}],"name":"mintTimelocked","outputs":[{"name":"","type":"address"}],"payable":false,"type":"function"},{"constant":true,"inputs":[{"name":"_owner","type":"address"},{"name":"_spender","type":"address"}],"name":"allowance","outputs":[{"name":"remaining","type":"uint256"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"payable":false,"type":"function"},{"anonymous":false,"inputs":[{"indexed":true,"name":"to","type":"address"},{"indexed":false,"name":"value","type":"uint256"}],"name":"Mint","type":"event"},{"anonymous":false,"inputs":[],"name":"MintFinished","type":"event"},{"anonymous":false,"inputs":[],"name":"Pause","type":"event"},{"anonymous":false,"inputs":[],"name":"Unpause","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"owner","type":"address"},{"indexed":true,"name":"spender","type":"address"},{"indexed":false,"name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"from","type":"address"},{"indexed":true,"name":"to","type":"address"},{"indexed":false,"name":"value","type":"uint256"}],"name":"Transfer","type":"event"}]')
# OMG ABI
address = '0xd26114cd6EE289AccF82350c8d8487fedB8A0C07'

contract = w3.eth.contract(address=address, abi=abi)

totalSupply = contract.functions.totalSupply().call()
print(w3.fromWei(totalSupply, 'ether'))
print(contract.functions.name().call())
print(contract.functions.symbol().call())
balance = contract.functions.balanceOf('0xd26114cd6EE289AccF82350c8d8487fedB8A0C07').call()
print(w3.fromWei(balance, 'ether'))

140245398.245132780789239631
OMGToken
OMG
26254.40668528280698004


In [ ]:
signed_tx = w3.eth.account.signTransaction(tx, private_key)

tx_hash = w3.eth.sendRawTransaction(signed_tx.rawTransaction)

print(w3.toHex(tx_hash))